In [1]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_excel('FraudDetectionData.xlsx')

In [3]:
df['Incident_minus_policy_date'] = df['incident_date'] - df['policy_bind_date']

In [4]:
df.loc[df['umbrella_limit'] < 0,['umbrella_limit']] = 0
df.loc[df['collision_type'] == '?',['collision_type']] = 'UNSURE'
df.loc[df['property_damage'] == '?',['property_damage']] = 'UNSURE'
df.loc[df['police_report_available'] == '?',['police_report_available']] = 'UNSURE'

In [5]:
df['incident_hour_of_the_day'].map(int)
def time_range(int):
    if int>=0 and int<6: return 'Midnight'
    elif int>=6 and int<12: return 'Morning'
    elif int>=12 and int<18: return 'Afternoon'
    else: return 'Night'
df['incident_time_of_the_day'] = df['incident_hour_of_the_day'].map(time_range)

In [6]:
df.drop(['incident_hour_of_the_day','policy_bind_date','insured_zip','capital-gains','capital-loss','incident_date','incident_location','_c39'], inplace=True, axis=1)

In [7]:
df.set_index('policy_number', inplace=True)

In [8]:
df['Incident_minus_policy_date'] = df['Incident_minus_policy_date'].dt.components.days

In [9]:
df

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,...,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,Incident_minus_policy_date,incident_time_of_the_day
policy_number,,,,,,,,,,,,,,,,,,,,,
521585,328,48,OH,250/500,1000,1406.91,0,MALE,MD,craft-repair,...,71610,6510,13020,52080,Saab,92x,2004,Y,100,Midnight
342868,228,42,IN,250/500,2000,1197.22,5000000,MALE,MD,machine-op-inspct,...,5070,780,780,3510,Mercedes,E400,2007,Y,3130,Morning
687698,134,29,OH,100/300,2000,1413.14,5000000,FEMALE,PhD,sales,...,34650,7700,3850,23100,Dodge,RAM,2007,N,5282,Morning
227811,256,41,IL,250/500,2000,1415.74,6000000,FEMALE,PhD,armed-forces,...,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,8996,Midnight
367455,228,44,IL,500/1000,1000,1583.91,6000000,MALE,Associate,sales,...,6500,1300,650,4550,Accura,RSX,2009,N,256,Night
104594,256,39,OH,250/500,1000,1351.10,0,FEMALE,PhD,tech-support,...,64100,6410,6410,51280,Saab,95,2003,Y,3004,Night
413978,137,34,IN,250/500,1000,1333.35,0,MALE,PhD,prof-specialty,...,78650,21450,7150,50050,Nissan,Pathfinder,2012,N,5336,Midnight
429027,165,37,IL,100/300,1000,1137.03,0,MALE,Associate,tech-support,...,51590,9380,9380,32830,Audi,A5,2015,N,9155,Night
485665,27,33,IL,100/300,500,1442.99,0,FEMALE,PhD,other-service,...,27700,2770,2770,22160,Toyota,Camry,2012,N,6568,Night


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [22]:
df_x=df.drop(["fraud_reported","auto_make","auto_model"], axis=1, inplace=False)
df_x=pd.get_dummies(df_x)
df_y=df["fraud_reported"]
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_x, df_y, test_size=0.3, random_state=1234)

print("train data X size:",df_train_x.shape)
print("train data Y size:",df_train_y.shape)
print("test data X size:",df_test_x.shape)
print("test data Y size:",df_test_y.shape)

('train data X size:', (700, 110))
('train data Y size:', (700L,))
('test data X size:', (300, 110))
('test data Y size:', (300L,))


In [23]:
rf_uncustomized = RandomForestClassifier(random_state=1234)
rf_uncustomized.fit(df_train_x, df_train_y)
print("Accuracy on training set: {:.3f}".format(rf_uncustomized.score(df_train_x,df_train_y)))
print("Accuracy on test set: {:.3f}".format(rf_uncustomized.score(df_test_x,df_test_y)))

Accuracy on training set: 0.987
Accuracy on test set: 0.773
